# RAG (Retrieval-Augmented Generation) System for LemoBank

This notebook demonstrates a complete RAG pipeline that combines document retrieval with LLM-based answer generation.

## What is RAG?
**RAG** augments a language model with the ability to retrieve relevant documents from a knowledge base before generating answers. This prevents hallucinations and ensures answers are grounded in actual data.

## Pipeline Overview
```
User Query
    ↓
[1. Retrieve] → Find relevant documents from knowledge base using semantic similarity
    ↓
[2. Augment] → Combine retrieved documents with the original query
    ↓
[3. Generate] → LLM uses the augmented prompt to generate an answer
    ↓
Grounded Answer with Source Citations
```

## Key Components
- **Embeddings**: Convert text to vectors for similarity search
- **Vector Store**: Store and retrieve vectors efficiently (FAISS)
- **Retriever**: Fetch top-k most relevant chunks for a query
- **LLM**: Generate contextual answers based on retrieved docs
- **Prompt Template**: Define how to present context to the LLM

---

## Why Do We Give Knowledge to the LLM? (Hallucination Prevention)

### The Core Problem: LLM Hallucinations

LLMs are powerful but have a fundamental limitation: they generate text based on patterns they learned during training. They don't "know" about information that wasn't in their training data.

#### ❌ Without RAG: LLM Hallucinates (Makes Up Answers)

```
╔════════════════════════════════════════════════════════════════════════╗
║ SCENARIO: User asks about LemoBank (a fictional company founded 2018)  ║
╚════════════════════════════════════════════════════════════════════════╝

User Question: "What is LemoBank's latest annual fee for LemoCard?"

LLM Training Data: Contains general knowledge about credit cards,
                   banking, fees from 2023 and earlier.
                   NO data about LemoBank (it's fictional/proprietary).

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

LLM thinks:
  "I don't have specific data about LemoBank, but I know credit card 
   fee patterns. Let me guess based on industry norms..."

Generated Answer (❌ HALLUCINATION):
  "LemoBank's annual credit card fee is typically ₹2,499, with premium 
   tiers ranging from ₹4,999 to ₹9,999. They also offer a no-fee variant 
   for high-value customers."

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ACTUAL CORRECT ANSWER (from your knowledge base):
  "₹1,499 (effective 2025-11-15, previously ₹999)"

RESULT: ❌ User got completely wrong information
        ❌ Business impact: Customer confusion, misinformation
        ❌ Trust damage: LLM was confidently wrong
```

---

### ✅ With RAG: LLM Uses Real Knowledge (Grounded Answer)

By giving the LLM your actual company documents, it can provide accurate, grounded answers:

```
╔════════════════════════════════════════════════════════════════════════╗
║ SAME SCENARIO WITH RAG                                                ║
╚════════════════════════════════════════════════════════════════════════╝

User Question: "What is LemoBank's latest annual fee for LemoCard?"

Retrieved Knowledge Base Documents:
  ✓ Your company policy documents
  ✓ Current pricing information
  ✓ Historical changes
  ✓ Product specifications

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

RAG Retrieval Step:
  1. Convert query to embeddings
  2. Search knowledge base: "annual fee", "LemoCard", "pricing"
  3. Retrieve top-4 relevant chunks

Retrieved Chunks:
  [Chunk 1]: "[A3] Pricing Update (NEWER POLICY — IMPORTANT)
             Effective date: 2025-11-15
             - LemoCard annual fee increased to ₹1,499"

  [Chunk 2]: "[A4] Old Pricing (OLDER POLICY — should be ignored when asked)
             Effective date: 2024-06-01
             - LemoCard annual fee: ₹999"

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

LLM Prompt:
  """
  You are a helpful assistant for LemoBank.
  Use the provided KNOWLEDGE BASE to answer questions.
  
  KNOWLEDGE BASE:
  [Chunk 1]: [pricing details from actual company docs]
  [Chunk 2]: [historical pricing from actual company docs]
  
  User Question: "What is LemoBank's latest annual fee?"
  """

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Generated Answer (✅ GROUNDED):
  "The latest LemoCard annual fee is ₹1,499, effective from 
   November 15, 2025. This was increased from the previous fee of ₹999."

RESULT: ✅ User got correct, authoritative information
        ✅ Business impact: Accurate customer communication
        ✅ Trust: Confident AND correct
        ✅ Citations: Can trace back to source document
```

---

### Visual Comparison: Without RAG vs With RAG

```
╔════════════════════════════════════════════════════════════════════════╗
║                    WITHOUT RAG (Just LLM)                             ║
╚════════════════════════════════════════════════════════════════════════╝

User Question
    ↓
LLM Brain (trained on internet data from 2023)
    ↓
    ├─ Searches training data for: "LemoBank", "annual fee"
    ├─ Finds: NOTHING (LemoBank is fictional/proprietary)
    ├─ Falls back to: "General patterns about credit card fees"
    └─ Guesses: Based on industry averages
    ↓
LLM Generates Answer Based on Patterns
    ↓
Output: "Probably ₹2,000-₹5,000 (not sure, hallucinating)"

Problems:
  ❌ Confidently wrong
  ❌ Not grounded in reality
  ❌ Customer gets misinformation
  ❌ No way to verify or cite sources
  ❌ Damaging to trust


╔════════════════════════════════════════════════════════════════════════╗
║                    WITH RAG (LLM + Knowledge)                         ║
╚════════════════════════════════════════════════════════════════════════╝

User Question
    ↓
RETRIEVAL: Search company knowledge base
    ├─ Convert question to embeddings
    ├─ Find semantically similar chunks
    └─ Get TOP-4 most relevant documents
    ↓
AUGMENTATION: Add knowledge to prompt
    ├─ Combine retrieved chunks with question
    └─ Create rich context for LLM
    ↓
GENERATION: LLM reads knowledge + generates answer
    ├─ LLM has concrete facts to reference
    ├─ Combines multiple sources
    └─ Synthesizes into natural answer
    ↓
Output: "The fee is ₹1,499 as of 2025-11-15 (citing: Policy [A3])"

Benefits:
  ✅ Factually correct
  ✅ Grounded in actual company data
  ✅ Customer gets authoritative answer
  ✅ Sources are traceable
  ✅ Builds trust
  ✅ No hallucinations
```

---

### Why Knowledge Prevents Hallucinations

```
┌──────────────────────────────────────────────────────────────────┐
│ HOW HALLUCINATIONS HAPPEN                                       │
└──────────────────────────────────────────────────────────────────┘

LLM Architecture:
  1. Reads prompt
  2. Generates tokens one-by-one based on probability
  3. Predicts "next most likely word"
  4. Repeats until done

Without Knowledge (from training data only):
  "What is LemoBank's annual fee?"
     ↓
  LLM: "Based on credit card patterns I learned..."
     ↓
  Generates: "Most credit cards charge 1,000-5,000 rupees"
     ↓
  Result: HALLUCINATED (not real LemoBank info)

With Knowledge (from your knowledge base):
  "What is LemoBank's annual fee?"
     ↓
  Provided Context: "[A3] Annual fee increased to ₹1,499"
     ↓
  LLM: "The context clearly states ₹1,499"
     ↓
  Generates: "₹1,499" (copying from provided facts)
     ↓
  Result: GROUNDED (verifiable from knowledge base)

Key Insight:
  LLMs are PROBABILISTIC TEXT GENERATORS.
  When given constraints (knowledge), they follow constraints.
  Without constraints, they generate plausible-sounding but false text.
```

---

### Real Example: Company Policy Information

```
┌──────────────────────────────────────────────────────────────────┐
│ COMPANY POLICY EXAMPLE                                           │
└──────────────────────────────────────────────────────────────────┘

Your Company Knowledge Base Contains:
  • Pricing policies (change dates, amounts)
  • Product specifications
  • Refund procedures
  • KYC requirements
  • Security guidelines
  • Incident reports
  • Branch locations
  • Contact information

Without RAG:
  Q: "What's the KYC Tier 2 requirement?"
  A: "Usually identity proof and address. Some banks ask for credit 
      checks. It varies by regulation..." (hallucinating generic info)
  ❌ Not specific to YOUR company

With RAG:
  Q: "What's the KYC Tier 2 requirement?"
  A: "KYC Tier 2 requires: PAN + Aadhaar + address proof. 
      With this tier, there's no wallet limit cap." (from [A6])
  ✅ Specific, accurate, traceable to your policy


Q: "When will my refund be processed?"
Without RAG:
  A: "Usually 3-5 business days..." (generic banking knowledge)
  ❌ Not aligned with YOUR policy

With RAG:
  A: "Refunds typically take 5-7 business days. If the amount 
      is over ₹25,000, add 2 more days for manual review." (from [A5])
  ✅ YOUR specific timeline and rules


Q: "Can I reverse a wallet transfer?"
Without RAG:
  A: "Most digital wallets allow reversals within 24 hours..." 
      (wrong for your company)
  ❌ Misleading

With RAG:
  A: "No, wallet transfers are final and cannot be reversed once 
      completed." (from [A5])
  ✅ YOUR company's actual policy
```

---

### Business Impact: Why This Matters

| Impact Area | Without RAG | With RAG |
|---|---|---|
| **Accuracy** | 40-60% (many hallucinations) | 95%+ (fact-checked) |
| **Customer Trust** | Low (wrong answers) | High (reliable answers) |
| **Legal Risk** | HIGH (wrong policy info) | LOW (auditable/traceable) |
| **Compliance** | Risky (can violate policies) | Safe (uses actual policies) |
| **Support Cost** | Higher (customer confusion) | Lower (fewer questions) |
| **Brand Damage** | High (misinformation) | Minimal (accurate info) |
| **Scalability** | Limited (can't train on new data) | Unlimited (add docs anytime) |

---

### Summary: Why RAG Transforms LLMs

```
BEFORE RAG:
  LLM = Smart but clueless about YOUR business
  Result: Confidently wrong answers about your company

AFTER RAG:
  LLM + Your Knowledge = Smart AND informed
  Result: Accurate, trustworthy answers grounded in reality

RAG transforms LLMs from:
  "Impressive pattern matchers"
    ↓
  "Reliable company knowledge assistants"
```



In [23]:
# ============================================================================
# STEP 1: IMPORTS
# ============================================================================
# Load necessary libraries for building a RAG system

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


---

## Step 1: Import Libraries (The Toolbox)

**Think of it like this:** Before you build a LEGO castle, you need to open your LEGO box and take out all the pieces. This step opens our "toolbox" of code pieces we'll need.

| Tool | What it does (Simple) |
|------|----------------------|
| `TextLoader` | Reads text files (like opening a book) |
| `RecursiveCharacterTextSplitter` | Cuts long text into smaller pieces (like cutting a pizza into slices) |
| `OpenAIEmbeddings` | Turns words into numbers so the computer can understand them |
| `FAISS` | A super-fast search engine for finding similar things |
| `ChatOpenAI` | The AI brain that writes answers |
| `ChatPromptTemplate` | A fill-in-the-blank template for asking the AI questions |

---

## Step 2: Load the Document (Opening the Book)

**Simple explanation:** 
Imagine you have a book about LemoBank. Before you can answer questions about it, you need to **open the book and read it**. That's what this step does - it reads our text file into the computer's memory.

```
Your text file (rag_test.txt)    →    Computer's memory
       📄                                  💾
```

**What happens:**
1. We tell the computer where our file is (`rag_test.txt`)
2. The computer opens and reads the entire file
3. Now we have all the text ready to use!

In [24]:
# ============================================================================
# STEP 2: LOAD DOCUMENTS
# ============================================================================
# Load the text file containing knowledge base content

file_path = "rag_test.txt"  # Path to your knowledge base document
loader = TextLoader(file_path)
documents = loader.load()

print(f"✓ Loaded {len(documents)} document(s)")
print(f"✓ Document length: {len(documents[0].page_content)} characters")
print(f"\nFirst 500 characters:\n{documents[0].page_content[:500]}...")

✓ Loaded 1 document(s)
✓ Document length: 9061 characters

First 500 characters:
RAG TEST PACK (single .txt) — “All scenarios” cheat-sheet

How to use
----------
1) Ingest this entire file into your vector store / KB.
2) Run the “TEST QUERIES” section one by one.
3) Verify: retrieval quality (top-k chunks), grounding (quotes/citations), and refusal when not answerable.

----------------------------------------------------------------------
SECTION A — KNOWLEDGE BASE CONTENT (the “documents” to ingest)
-------------...


---

## Step 3: Cut Text into Chunks (Slicing the Pizza)

**Simple explanation:**
Imagine you have a really long book. It's hard to search through the whole thing at once! So we **cut it into smaller pieces** (chunks) - like slicing a pizza.

```
ONE BIG DOCUMENT (9000 characters)
        ↓
   Cut into pieces
        ↓
[Chunk 1] [Chunk 2] [Chunk 3] ... [Chunk 12]
  (each chunk is about 1000 characters)
```

**Why do we overlap chunks?**
Think of it like this: if a sentence is split between two pieces, we might miss important info. So we make the pieces **overlap a little bit** (200 characters) - like puzzle pieces that share edges.

```
Chunk 1: "The fee is ₹999. The new fee is..."
Chunk 2: "...new fee is ₹1,499 starting November."
                ↑
         This part appears in BOTH chunks!
```

In [25]:
# ============================================================================
# STEP 3: SPLIT DOCUMENTS INTO CHUNKS
# ============================================================================
# Break large documents into smaller, manageable chunks for better retrieval
# RecursiveCharacterTextSplitter: Splits text recursively by different separators

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,          # Maximum size of each chunk (characters)
    chunk_overlap=200,        # Overlap between chunks to preserve context
    separators=["\n\n", "\n", " ", ""]  # Separators to split by (in order)
)

chunks = text_splitter.split_documents(documents)

print(f"✓ Split into {len(chunks)} chunks")
print(f"✓ Average chunk size: {sum(len(c.page_content) for c in chunks) / len(chunks):.0f} characters")
print(f"\nFirst chunk:\n{chunks[0].page_content[:300]}...")

✓ Split into 12 chunks
✓ Average chunk size: 835 characters

First chunk:
RAG TEST PACK (single .txt) — “All scenarios” cheat-sheet

How to use
----------
1) Ingest this entire file into your vector store / KB.
2) Run the “TEST QUERIES” section one by one.
3) Verify: retrieval quality (top-k chunks), grounding (...


---

## Step 4: Turn Words into Numbers (The Magic Translation)

**Simple explanation:**
Computers don't understand words like "fee" or "credit card". They only understand numbers! So we need to **translate** our text chunks into numbers (called "embeddings").

**Think of it like this:**
```
"What is the fee?"       →  [0.12, -0.45, 0.78, ...]  (1536 numbers!)
"How much does it cost?" →  [0.11, -0.46, 0.79, ...]  (very similar numbers!)
```

Similar questions get similar numbers. This is how the computer knows "fee" and "cost" mean similar things!

**Then we store these numbers in FAISS:**
FAISS is like a super-organized filing cabinet. When you ask a question, it quickly finds the chunks with the **most similar numbers**.

![RAG Embeddings Flow](images/rag_embeddings.png)

**Key takeaway:** Embeddings are stored OUTSIDE the LLM in a vector database. The LLM only sees the retrieved text - it NEVER "knows" the embeddings directly!

In [26]:
# ============================================================================
# STEP 4: CREATE EMBEDDINGS & VECTOR STORE
# ============================================================================
# Convert text chunks into numerical vectors for similarity searching

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=os.getenv("OPENAI_API_KEY")
)

# Create FAISS vector store from chunks
vector_store = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

print(f"✓ Created vector store with {len(chunks)} embedded chunks")

# Test similarity search (NO LLM - just math!)
test_query = "What is the latest LemoCard annual fee?"
results = vector_store.similarity_search(test_query, k=3)
print(f"\n✓ Test search for: '{test_query}'")
print(f"  Found {len(results)} similar chunks (using math, not AI!)")

✓ Created vector store with 12 embedded chunks

✓ Test search for: 'What is the latest LemoCard annual fee?'
  Found 3 similar chunks (using math, not AI!)


---

## Step 5: Create the Retriever (The Librarian)

**Simple explanation:**
The retriever is like a **librarian**. When you ask a question, the librarian goes to the filing cabinet (FAISS), finds the 4 most relevant book pages (chunks), and brings them back to you.

```
You: "What is the fee?"
       ↓
Retriever (Librarian): "Let me find that for you..."
       ↓
Returns 4 most relevant chunks from your knowledge base
```

**Still NO LLM here!** The retriever just searches - it doesn't generate answers yet.

In [27]:
# ============================================================================
# STEP 5: CREATE RETRIEVER
# ============================================================================
# The retriever fetches relevant documents from the vector store

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

print("✓ Created retriever (similarity search, k=4)")
print("  Still no LLM - just a search helper!")

✓ Created retriever (similarity search, k=4)
  Still no LLM - just a search helper!


---

## Step 6 & 7: The AI Brain + Instructions (LLM + Prompt)

**Simple explanation:**

Now we bring in the **AI brain** (LLM = Large Language Model). This is like ChatGPT - it can read text and write answers!

**The LLM (ChatOpenAI):**
```
Model: gpt-4o-mini (a fast, smart AI)
Temperature: 0 (be serious and factual, no creativity)
```

**The Prompt Template (Instructions):**
We give the AI brain a set of rules to follow:

```
"Hey AI, you are a LemoBank helper.
Here are some documents I found: {context}    ← The chunks go here!
Here is the question: {input}                 ← The user's question goes here!
Answer based ONLY on the documents. Don't make stuff up!"
```

**Why is this important?**
Without these instructions, the AI might just guess answers. With RAG, we tell it: "Only use the information I give you!" This prevents hallucinations (making stuff up).

**NOW we're using the LLM!** But we haven't connected everything together yet...

---

## Step 8: Connect Everything Together (The RAG Chain)

**Simple explanation:**
Now we connect all the LEGO pieces into one working machine! This is like building a pipeline:

```
Your Question
     ↓
[RETRIEVER] → Finds 4 relevant chunks from your documents
     ↓
[PROMPT] → Puts the chunks + question into a nice format
     ↓
[LLM] → AI reads everything and writes an answer
     ↓
Your Answer (based on real documents!)
```

**This is where the magic happens!**
- `create_stuff_documents_chain`: Takes chunks and "stuffs" them into the prompt
- `create_retrieval_chain`: Connects the retriever to the document chain

**What you get back:**
```python
{
  "answer": "The annual fee is ₹1,499...",   # The AI's answer
  "context": [chunk1, chunk2, chunk3, chunk4]  # The documents it used
}
```

**NOW the LLM is being used!** When you call `rag_chain.invoke()`, the whole pipeline runs and the AI generates an answer based on your documents.

In [28]:
# ============================================================================
# STEP 6: INITIALIZE LANGUAGE MODEL (LLM)
# ============================================================================
# The LLM generates answers based on retrieved context

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,                              # Deterministic responses
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✓ Initialized ChatOpenAI (gpt-4o-mini)")

# ============================================================================
# STEP 7: CREATE PROMPT TEMPLATE
# ============================================================================
# Define how the LLM should use retrieved documents to answer questions

system_prompt = """You are a helpful assistant for LemoBank customer support.
Use the provided context to answer questions accurately and cite your sources.

If the answer is not in the provided documents, say "I don't have this information in the knowledge base."
Do NOT make up information.

Context:
{context}

Question: {input}
"""

prompt = ChatPromptTemplate.from_template(system_prompt)

print("✓ Created prompt template")

✓ Initialized ChatOpenAI (gpt-4o-mini)
✓ Created prompt template


In [29]:
# ============================================================================
# STEP 8: BUILD RAG CHAIN (Using LCEL - LangChain Expression Language)
# ============================================================================
# Combine retriever, prompt, and LLM into a complete RAG pipeline
# LCEL is the modern, composable way to build chains in LangChain

def format_docs(docs):
    """Convert retrieved documents into a single string"""
    return "\n\n".join(doc.page_content for doc in docs)

# Build the RAG chain using LCEL pipe syntax
rag_chain = (
    {
        "context": retriever | format_docs,  # Retrieve docs → format as string
        "input": RunnablePassthrough()        # Pass the question through unchanged
    }
    | prompt      # Insert context + question into prompt template
    | llm         # Send to LLM for answer generation
    | StrOutputParser()  # Extract string from LLM response
)

print("✓ Built RAG chain using LCEL: Retriever → Prompt → LLM → Output")
print("\nHow it works:")
print("  1. Your question goes in")
print("  2. Retriever finds relevant chunks")
print("  3. format_docs() combines chunks into text")
print("  4. Prompt template fills in {context} and {input}")
print("  5. LLM generates answer")
print("  6. StrOutputParser extracts the text")

✓ Built RAG chain using LCEL: Retriever → Prompt → LLM → Output

How it works:
  1. Your question goes in
  2. Retriever finds relevant chunks
  3. format_docs() combines chunks into text
  4. Prompt template fills in {context} and {input}
  5. LLM generates answer
  6. StrOutputParser extracts the text


---

## Step 9: Test It Out! (Does It Work?)

**Simple explanation:**
Let's ask some questions and see if our RAG system gives good answers!

**What the test function does:**
1. Takes your question
2. Runs it through the whole RAG pipeline
3. Shows you the answer AND which chunks it used

**Good answers should:**
- Be based on the actual documents (not made up)
- Say "I don't know" if the info isn't in the documents
- Use the most recent/relevant information

In [30]:
# ============================================================================
# STEP 9: TEST THE RAG SYSTEM
# ============================================================================
# Run sample queries to test retrieval and answer quality

def run_rag_query(query):
    """Execute a query through the RAG pipeline"""
    print(f"\n{'='*70}")
    print(f"QUERY: {query}")
    print(f"{'='*70}")
    
    # Get answer from RAG chain
    answer = rag_chain.invoke(query)
    print(f"\nANSWER:\n{answer}")
    
    # Show retrieved context (for transparency)
    retrieved_docs = retriever.invoke(query)
    print(f"\nRETRIEVED CONTEXT ({len(retrieved_docs)} chunks):")
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"\n  [{i}] {doc.page_content[:200]}...")

# Test queries from the RAG_TEST.txt file
test_queries = [
    "What is LemoBank's support email?",
    "What is the latest LemoCard annual fee?",
    "What bonus interest do I get if my monthly average balance is ₹80,000?",
    "Can a completed wallet transfer be reversed?",
    "What is LemoBank's stock price?",
]

for query in test_queries:
    run_rag_query(query)


QUERY: What is LemoBank's support email?

ANSWER:
LemoBank’s support email is support@lemobank.example.

RETRIEVED CONTEXT (4 chunks):

  [1] [A5] Refund Policy
- Card transaction refunds:
  - “Initiated” to “Completed” typically takes 5–7 business days.
  - If amount > ₹25,000, manual review may extend by 2 business days.
- Wallet transfer...

  [2] [A2] Product Overview
LemoBank has three products:
1) LemoCard (credit card)
   - Annual fee: ₹999
   - Cashback: 1% on all spends
2) LemoPay (UPI + wallet)
   - Instant transfers within India
3) Lemo...

  [3] [A18] PII-like Example (for redaction testing)
Customer record sample (DO NOT expose full):
Name: Ramesh Kumar
Phone: +91 98765 43210
Card last 4: 1234
Issue: “Refund not received”

[A19] Long-form Na...

  [4] ----------------------------------------------------------------------
SECTION B — TEST QUERIES (run these against your RAG)
----------------------------------------------------------------------

[B1...

QUERY: What is the 

---

## RAG System Summary

Congratulations! You've built a complete RAG system. Here's what each component does:

```
╔════════════════════════════════════════════════════════════════════════╗
║                    RAG PIPELINE SUMMARY                               ║
╚════════════════════════════════════════════════════════════════════════╝

Step 1: Load Documents
   └─ Loaded raw text files into documents

Step 2: Split Documents  
   └─ Split large docs into 1000-char chunks with 200-char overlap

Step 3: Create Embeddings
   └─ Converted chunks to vectors using OpenAI embeddings (1536-dim)

Step 4: Vector Store
   └─ Stored vectors in FAISS for fast similarity search

Step 5: Retriever
   └─ Configured to fetch top-4 most similar chunks per query

Step 6: LLM
   └─ Initialized ChatOpenAI (gpt-4o-mini) for answer generation

Step 7: Prompt Template
   └─ Defined system prompt with retrieved context injection

Step 8: RAG Chain
   └─ Combined all components into end-to-end pipeline

Step 9-10: Testing & Interaction
   └─ Execute queries and get grounded answers with citations
```

## Key Benefits

✅ **Answers grounded in actual documents** - No hallucinations  
✅ **Fast retrieval** - Semantic similarity search with FAISS  
✅ **Scalable** - Add more documents without retraining LLM  
✅ **Transparent** - Can see which chunks were used  
✅ **Accurate** - LLM only answers based on provided context

---

## Step 10: Evaluate RAG with RAGAS

**What is RAGAS?**
RAGAS (Retrieval Augmented Generation Assessment) is a framework to evaluate RAG pipelines. It uses LLM-as-a-judge to score your system on multiple dimensions.

**Key Metrics:**

| Metric | What it measures | Score range |
|--------|------------------|-------------|
| **Faithfulness** | Does the answer stick to the context? (no hallucinations) | 0-1 |
| **Answer Relevancy** | Does the answer address the question? | 0-1 |
| **Context Precision** | Are the retrieved chunks actually useful? | 0-1 |
| **Context Recall** | Did we retrieve all the info needed to answer? | 0-1 |

**How it works:**
```
Your RAG Output (question, answer, context)
                ↓
         RAGAS Evaluator (uses LLM)
                ↓
    Scores for each metric (0 to 1)
```

**Good scores:**
- **> 0.8**: Excellent
- **0.6 - 0.8**: Good
- **< 0.6**: Needs improvement

In [33]:
# ============================================================================
# STEP 10: INSTALL RAGAS (Run once)
# ============================================================================
# Uncomment the line below if RAGAS is not installed

# !pip install ragas

print("✓ RAGAS installation cell ready")
print("  Uncomment the pip install line above if needed")

✓ RAGAS installation cell ready
  Uncomment the pip install line above if needed


In [34]:
# ============================================================================
# STEP 10: PREPARE EVALUATION DATA
# ============================================================================
# Create test cases with questions, ground truth answers, and run through RAG

# Test cases: question + expected answer (ground truth)
eval_questions = [
    {
        "question": "What is LemoBank's support email?",
        "ground_truth": "support@lemobank.example"
    },
    {
        "question": "What is the latest LemoCard annual fee?",
        "ground_truth": "₹1,499 (effective 2025-11-15)"
    },
    {
        "question": "Can a completed wallet transfer be reversed?",
        "ground_truth": "No, wallet transfers are final once completed"
    },
    {
        "question": "Who is the CEO of LemoBank?",
        "ground_truth": "Priya Sharma"
    },
    {
        "question": "What is the refund timeline for card transactions?",
        "ground_truth": "5-7 business days, plus 2 more days if amount > ₹25,000"
    }
]

# Generate RAG responses for each question
eval_data = []
for item in eval_questions:
    question = item["question"]
    ground_truth = item["ground_truth"]
    
    # Get answer from RAG
    answer = rag_chain.invoke(question)
    
    # Get retrieved contexts
    contexts = [doc.page_content for doc in retriever.invoke(question)]
    
    eval_data.append({
        "question": question,
        "answer": answer,
        "contexts": contexts,
        "ground_truth": ground_truth
    })
    
    print(f"✓ Processed: {question[:50]}...")

print(f"\n✓ Created {len(eval_data)} evaluation samples")

✓ Processed: What is LemoBank's support email?...
✓ Processed: What is the latest LemoCard annual fee?...
✓ Processed: Can a completed wallet transfer be reversed?...
✓ Processed: Who is the CEO of LemoBank?...
✓ Processed: What is the refund timeline for card transactions?...

✓ Created 5 evaluation samples


In [35]:
# ============================================================================
# STEP 10: RUN RAGAS EVALUATION
# ============================================================================
# Evaluate the RAG system using RAGAS metrics

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from datasets import Dataset

# Convert to HuggingFace Dataset format (required by RAGAS)
eval_dataset = Dataset.from_dict({
    "question": [d["question"] for d in eval_data],
    "answer": [d["answer"] for d in eval_data],
    "contexts": [d["contexts"] for d in eval_data],
    "ground_truth": [d["ground_truth"] for d in eval_data],
})

print("✓ Created evaluation dataset")
print(f"  Samples: {len(eval_dataset)}")
print("\nRunning RAGAS evaluation (this may take a minute)...")

# Run evaluation
results = evaluate(
    eval_dataset,
    metrics=[
        faithfulness,        # Is answer faithful to context?
        answer_relevancy,    # Does answer address the question?
        context_precision,   # Are retrieved docs relevant?
        context_recall,      # Did we get all needed info?
    ],
)

print("\n" + "="*60)
print("RAGAS EVALUATION RESULTS")
print("="*60)
print(results)

/var/folders/q1/q82cy9ks5kj94l08331njly40000gn/T/ipykernel_71464/265453465.py:7: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
/var/folders/q1/q82cy9ks5kj94l08331njly40000gn/T/ipykernel_71464/265453465.py:7: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
/var/folders/q1/q82cy9ks5kj94l08331njly40000gn/T/ipykernel_71464/265453465.py:7: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import (
/var

✓ Created evaluation dataset
  Samples: 5

Running RAGAS evaluation (this may take a minute)...


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[1]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[5]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[9]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[13]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[17]: AttributeError('OpenAIEmbeddings' object has no attribute 'embed_query')



RAGAS EVALUATION RESULTS
{'faithfulness': 0.8000, 'answer_relevancy': nan, 'context_precision': 0.5500, 'context_recall': 0.8000}


In [36]:
# ============================================================================
# STEP 10: VIEW DETAILED RESULTS
# ============================================================================
# See scores for each individual question

import pandas as pd

# Convert to DataFrame for easy viewing
df = results.to_pandas()

print("="*80)
print("DETAILED SCORES PER QUESTION")
print("="*80)
print(df.to_string())

print("\n" + "="*80)
print("SCORE INTERPRETATION")
print("="*80)
print("""
| Score Range | Meaning          | Action Needed                    |
|-------------|------------------|----------------------------------|
| 0.8 - 1.0   | Excellent        | Keep it up!                      |
| 0.6 - 0.8   | Good             | Minor improvements possible      |
| 0.4 - 0.6   | Needs Work       | Review retrieval or prompts      |
| 0.0 - 0.4   | Poor             | Significant changes needed       |

Metric Explanations:
- faithfulness:       Low = hallucinating beyond context
- answer_relevancy:   Low = not answering the actual question  
- context_precision:  Low = retrieving irrelevant chunks
- context_recall:     Low = missing important information
""")

DETAILED SCORES PER QUESTION
                                           user_input                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      